In [2]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import sys


# Caminho absoluto até a raiz do projeto (pasta que contém 'nlp')
ROOT_DIR = os.path.abspath('C:/Users/marci/OneDrive/Documentos/IFPE - (ADS)/aulas/IFPE_TADS/Semestre II/engenharia_softwares/tads_eng_softw')
sys.path.insert(0, ROOT_DIR)
base_path = Path('C:/Users/marci/OneDrive/Documentos/IFPE - (ADS)/aulas/IFPE_TADS/Semestre II/engenharia_softwares/tads_eng_softw/nlp/ml/database/datasets/bases_geradas')
# Agora sim, importa o módulo
from nlp.config.connection import get_connection_ml



pd.set_option('display.max_columns', None)   # Mostra todas as colunas
pd.set_option('display.width', None)         # Não quebra linha automaticamente
pd.set_option('display.max_colwidth', None)  # Mostra todo o conteúdo das células

## Listar os arquivos CSV na Pasta

In [3]:
csv_files = list(base_path.rglob("*.csv"))

if not csv_files:
    print("Nenhum arquivo CSV encontrado.")
else:
    for csv_file in csv_files:
        print(f"Arquivo: {csv_file}")
        # df = pd.read_csv(csv_file)
        # display(df.head())  

Arquivo: C:\Users\marci\OneDrive\Documentos\IFPE - (ADS)\aulas\IFPE_TADS\Semestre II\engenharia_softwares\tads_eng_softw\nlp\ml\database\datasets\bases_geradas\avaliacoes_ironicas_60k.csv
Arquivo: C:\Users\marci\OneDrive\Documentos\IFPE - (ADS)\aulas\IFPE_TADS\Semestre II\engenharia_softwares\tads_eng_softw\nlp\ml\database\datasets\bases_geradas\avaliacoes_negativas_60k.csv
Arquivo: C:\Users\marci\OneDrive\Documentos\IFPE - (ADS)\aulas\IFPE_TADS\Semestre II\engenharia_softwares\tads_eng_softw\nlp\ml\database\datasets\bases_geradas\avaliacoes_neutras_60k.csv


### Arquivo Dados Classificados por notas BuscaPe

#### Base de Dados do Kaggle
. https://www.kaggle.com/datasets/fredericods/ptbr-sentiment-analysis-datasets?utm_source=chatgpt.com&select=buscape.csv

In [12]:
base_dados = next(Path(base_path).rglob("avaliacoes_neutras_60k.csv"))

df = pd.read_csv(base_dados)
df.head(5)

,contexto_pergunta,conteudo_resposta,classificada_como_sentimento
0,Como você avalia o produto que recebeu?,"Sem problemas, tudo ok.",neutro
1,Como você avalia o produto que recebeu?,"Funciona normalmente, como esperado.",neutro
2,Como você avalia o produto que recebeu?,"Parece funcional, mas nada demais.",neutro
3,Como você avalia o produto que recebeu?,Recebi o produto conforme o anunciado.,neutro
4,Como você avalia o produto que recebeu?,"Sem problemas, tudo ok.",neutro


### Transformação na Tabela

In [13]:
## Ordenar as colunas
ordem_colunas = [
    'contexto_pergunta', 
    'conteudo_resposta',
    'classificada_como_sentimento',
]

df_transformado = df[ordem_colunas]

In [14]:
df_transformado.head(5)

,contexto_pergunta,conteudo_resposta,classificada_como_sentimento
0,Como você avalia o produto que recebeu?,"Sem problemas, tudo ok.",neutro
1,Como você avalia o produto que recebeu?,"Funciona normalmente, como esperado.",neutro
2,Como você avalia o produto que recebeu?,"Parece funcional, mas nada demais.",neutro
3,Como você avalia o produto que recebeu?,Recebi o produto conforme o anunciado.,neutro
4,Como você avalia o produto que recebeu?,"Sem problemas, tudo ok.",neutro


### Gerando Inserts para o Banco

In [15]:
# 1️⃣ Conexão com SQL Server
conn = get_connection_ml()
cursor = conn.cursor()

# 2️⃣ Montagem das linhas
linhas_insert = []

for index, row in df_transformado.iterrows():
    contexto = str(row['contexto_pergunta']).replace("'", "''")
    resposta = str(row['conteudo_resposta']).replace("'", "''")
    sentimento = str(row['classificada_como_sentimento']).replace("'", "''")

    linha = f"('{contexto}', '{resposta}', '{sentimento}')"
    linhas_insert.append(linha)

# 3️⃣ Quebrar em blocos de até 1000 e executar
blocos = [linhas_insert[i:i+1000] for i in range(0, len(linhas_insert), 1000)]

for bloco in blocos:
    valores_sql = ",\n".join(bloco)
    comando_full = f"""INSERT INTO Treinamento_Sentimento (
        contexto_pergunta, conteudo_resposta, classificada_como_sentimento
    ) VALUES
    {valores_sql};"""

    cursor.execute(comando_full)
    conn.commit()  # Aplica as inserções no banco

# 4️⃣ Encerrar conexão
# cursor.close()
# conn.close()
